# Read original datasets

Code adapted from Scanorama 

https://github.com/brianhie/scanorama 

In [1]:
import gzip
import os.path
import scipy.sparse
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix
from sklearn.preprocessing import normalize
import sys
import numpy as np


MIN_TRANSCRIPTS = 600

def load_tab(fname, max_genes=40000):
    if fname.endswith('.gz'):
        opener = gzip.open
    else:
        opener = open
        
    with opener(fname, 'r') as f:
        if fname.endswith('.gz'):
            header = f.readline().decode('utf-8').rstrip().split('\t')
        else:
            header = f.readline().rstrip().split('\t')
            
        cells = header[1:]
        X = np.zeros((len(cells), max_genes))
        genes = []
        for i, line in enumerate(f):
            if i > max_genes:
                break
            if fname.endswith('.gz'):
                line = line.decode('utf-8')
            fields = line.rstrip().split('\t')
            genes.append(fields[0])
            X[:, i] = [ float(f) for f in fields[1:] ]
    return X[:, range(len(genes))], np.array(cells), np.array(genes)

def load_mtx(dname):
    with open(dname + '/matrix.mtx', 'r') as f:
        while True:
            header = f.readline()
            if not header.startswith('%'):
                break
        header = header.rstrip().split()
        n_genes, n_cells = int(header[0]), int(header[1])

        data, i, j = [], [], []
        for line in f:
            fields = line.rstrip().split()
            data.append(float(fields[2]))
            i.append(int(fields[1])-1)
            j.append(int(fields[0])-1)
        X = csr_matrix((data, (i, j)), shape=(n_cells, n_genes))

    genes = []
    with open(dname + '/genes.tsv', 'r') as f:
        for line in f:
            fields = line.rstrip().split()
            genes.append(fields[1])
    assert(len(genes) == n_genes)

    return X, np.array(genes)

def process_tab(fname, min_trans=MIN_TRANSCRIPTS):
    X, cells, genes = load_tab(fname)

    gt_idx = [ i for i, s in enumerate(np.sum(X != 0, axis=1))
               if s >= min_trans ]
    X = X[gt_idx, :]
    cells = cells[gt_idx]
    if len(gt_idx) == 0:
        print('Warning: 0 cells passed QC in {}'.format(fname))
    if fname.endswith('.txt'):
        cache_prefix = '.'.join(fname.split('.')[:-1])
    elif fname.endswith('.txt.gz'):
        cache_prefix = '.'.join(fname.split('.')[:-2])
    elif fname.endswith('.tsv'):
        cache_prefix = '.'.join(fname.split('.')[:-1])
    elif fname.endswith('.tsv.gz'):
        cache_prefix = '.'.join(fname.split('.')[:-2])
    else:
        sys.stderr.write('Tab files should end with ".txt" or ".tsv"\n')
        exit(1)
        
    cache_fname = cache_prefix + '.npz'
    #np.savez(cache_fname, X=X, genes=genes)

    return X, cells, genes

def process_mtx(dname, min_trans=MIN_TRANSCRIPTS):
    X, genes = load_mtx(dname)

    gt_idx = [ i for i, s in enumerate(np.sum(X != 0, axis=1))
               if s >= min_trans ]
    X = X[gt_idx, :]
    if len(gt_idx) == 0:
        print('Warning: 0 cells passed QC in {}'.format(dname))
    
    cache_fname = dname + '/tab.npz'
    #scipy.sparse.save_npz(cache_fname, X, compressed=False)

    #with open(dname + '/tab.genes.txt', 'w') as of:
        #of.write('\n'.join(genes) + '\n')

    return X, genes

In [2]:
def read_real_data(counter):

    # Assume real datasets have cell as rows and genes as columns No index 
    # keep track of real data mappings 
    datamapping = { 1:'pancreas_inDrop', 2:'pancreas_multi_celseq2_expression_matrix',
                    4:'pancreas_multi_fluidigmc1_expression_matrix', 5:'pancreas_multi_smartseq2_expression_matrix',
                    3:'pancreas_multi_celseq_expression_matrix',
                    7:'jurkat', 6:'293t', 8:'jurkat_293t_50_50', 9:'jurkat_293t_99_1',
                    10:'68k_pbmc', 11:'b_cells', 12:'cd14_monocytes', 13:'cd4_t_helper', 14:'cd56_nk',
                    15:'cytotoxic_t', 16:'memory_t', 17:'regulatory_t', 18:'pbmc_10X', 19:'pbmc_kang',
                    20:'nuclei', 21:'Cerebellum_ALT', 22:'Cortex_noRep5_FRONTALonly', 23:'Cortex_noRep5_POSTERIORonly',
                    24:'EntoPeduncular', 25:'GlobusPallidus', 26:'Hippocampus', 27:'Striatum',
                    28:'SubstantiaNigra', 29:'Thalamus', 30:'GSM3589406_PP001swap.filtered.matrix',
                    31:'GSM3589407_PP002swap.filtered.matrix', 32:'GSM3589408_PP003swap.filtered.matrix',
                    33:'GSM3589409_PP004swap.filtered.matrix'}
    
    # 
    datasetname = datamapping[counter]

    ## TODO: Map to a sequence of related datasets in sparse matrix format

    if counter in [1,2,3,4,5]:
        genecountdata, cell_array, gene_array = process_tab('../Real/scanorama/pancreas/{}.txt.gz'.format(datasetname))
        genecountdata = csr_matrix(genecountdata)
        print('Read real dataset {}'.format(datasetname))    

    if counter in [6,7,8,9]:
        genecountdata, gene_array = process_mtx('../Real/scanorama/293t_jurkat/{}'.format(datasetname))
        genecountdata = csr_matrix(genecountdata)
        print('Read real dataset {}'.format(datasetname))   

    if counter in [10,11,12,13,14,15,16,17]:
        genecountdata, gene_array = process_mtx('../Real/scanorama/pbmc/10x/{}'.format(datasetname))
        genecountdata = csr_matrix(genecountdata)
        print('Read real dataset {}'.format(datasetname))   

    if counter in [18,19]:
        genecountdata, cell_array, gene_array = process_tab('../Real/scanorama/pbmc/{}.txt.gz'.format(datasetname))
        genecountdata = csr_matrix(genecountdata)
        print('Read real dataset {}'.format(datasetname))

    if counter in [20]:
        genecountdata, cell_array, gene_array = process_mtx('../Real/scanorama/mouse_brain/{}'.format(datasetname))
        genecountdata = csr_matrix(genecountdata)
        print('Read real dataset {}'.format(datasetname))

    if counter in [21,22,23,24,25,26,27,28,29]:
        genecountdata, cell_array, gene_array = process_mtx('../Real/scanorama/mouse_brain/dropviz/{}'.format(datasetname))
        genecountdata = csr_matrix(genecountdata)
        print('Read real dataset {}'.format(datasetname))

    if counter in [30,31,32,33,34]:
        genecountdata, cell_array, gene_array = process_tab('../Real/Levitin_bloodT/{}.txt.gz'.format(datasetname))
        genecountdata = csr_matrix(genecountdata)
        print('Read real dataset {}'.format(datasetname))


    return datasetname, genecountdata, gene_array

In [3]:
# Put datasets into a single matrix with the intersection of all genes.
def merge_datasets(datasets, genes, ds_names=None, verbose=True, union=False):
    if union:
        sys.stderr.write(
            "WARNING: Integrating based on the union of genes is "
            "highly discouraged, consider taking the intersection "
            "or requantifying gene expression.\n"
        )

    # Find genes in common.
    keep_genes = set()
    for idx, gene_list in enumerate(genes):
        if len(keep_genes) == 0:
            keep_genes = set(gene_list)
        elif union:
            keep_genes |= set(gene_list)
        else:
            keep_genes &= set(gene_list)
        if not union and not ds_names is None and verbose:
            print("After {}: {} genes".format(ds_names[idx], len(keep_genes)))
        if len(keep_genes) == 0:
            print("Error: No genes found in all datasets, exiting...")
            exit(1)
    if verbose:
        print("Found {} genes among all datasets".format(len(keep_genes)))

    if union:
        union_genes = sorted(keep_genes)
        for i in range(len(datasets)):
            if verbose:
                print("Processing data set {}".format(i))
            X_new = np.zeros((datasets[i].shape[0], len(union_genes)))
            X_old = csc_matrix(datasets[i])
            gene_to_idx = {gene: idx for idx, gene in enumerate(genes[i])}
            for j, gene in enumerate(union_genes):
                if gene in gene_to_idx:
                    X_new[:, j] = X_old[:, gene_to_idx[gene]].toarray().flatten()
            datasets[i] = csr_matrix(X_new)
        ret_genes = np.array(union_genes)
    else:
        # Only keep genes in common.
        ret_genes = np.array(sorted(keep_genes))
        for i in range(len(datasets)):
            # Remove duplicate genes.
            uniq_genes, uniq_idx = np.unique(genes[i], return_index=True)
            datasets[i] = datasets[i].tocsc()[:, uniq_idx]
            # Do gene filtering.
            gene_sort_idx = np.argsort(uniq_genes)
            gene_idx = [idx for idx in gene_sort_idx if uniq_genes[idx] in keep_genes]
            datasets[i] = datasets[i][:, gene_idx].tocsr()
            assert np.array_equal(uniq_genes[gene_idx], ret_genes)

    return datasets, ret_genes


# Create AnnData objects 

In [4]:
from anndata import AnnData

import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import scanpy as sc
from scipy.stats import entropy

In [5]:
integrationmapping = {
        1: "10Xmouse",
        2: "humanpancreas",
        3: "10Xpbmc",
    }

datamapping = {
    1: "pancreas_inDrop",
    2: "pancreas_multi_celseq2_expression_matrix",
    3: "pancreas_multi_celseq_expression_matrix",
    4: "pancreas_multi_fluidigmc1_expression_matrix",
    5: "pancreas_multi_smartseq2_expression_matrix",
    6: "jurkat_293t_50_50",
    7: "jurkat",
    8: "293t",
    9: "jurkat_293t_99_1",
    10: "68k_pbmc",
    11: "b_cells",
    12: "cd14_monocytes",
    13: "cd4_t_helper",
    14: "cd56_nk",
    15: "cytotoxic_t",
    16: "memory_t",
    17: "regulatory_t",
    18: "pbmc_10X",
    19: "pbmc_kang",
}

optionmapping = {
    1: [6, 7, 8],
    2: [1, 2, 3, 4, 5],
    3: [11, 12, 13, 14, 15, 16, 17, 18],
}

def batch_labels(Results,counter):
    dataset_labels = []
    no_datasets = len (Results['CountMatrix'])
    for i in range(0,no_datasets):
        datasetsize = Results['CountMatrix'][i].shape[0]
        for j in range(0,datasetsize):
            dataset_labels.append(datamapping[optionmapping[counter][i]])
    return dataset_labels

In [6]:
def read_cell_labels(counter):

    from sklearn.preprocessing import LabelEncoder
    
    integrationmapping = {
        2: "humanpancreas",
        1: "10Xmouse",
        3: "10Xpbmc",
    }
    

    if counter <= 5:
        cell_labels = (
            open("../Real/scanorama/cell_labels/{}_cell_labels.txt".format(integrationmapping[counter]))
            .read()
            .rstrip()
            .split()
        )

    if counter <= 5:
        le = LabelEncoder().fit(cell_labels)
        new_cell_labels = le.transform(cell_labels)
        cell_types = le.classes_
        print("There are {} types of cells".format(len(cell_types)))
        print(cell_types)

    return cell_labels, new_cell_labels, len(cell_types)

In [ ]:
for i in range(1,4):
    read_cell_labels(i)

In [ ]:
integrationmapping = {
        2: "humanpancreas",
        1: "10Xmouse",
        3: "10Xpbmc",
}


In [ ]:
for datasetnumber in range(2,4):
    casename = integrationmapping[datasetnumber]

    # Count Dataset 
    genecountdatalist = list()
    gene_arraylist = list()
    batch_labels = list()
    datamapping = { 1:'pancreas_inDrop', 2:'pancreas_multi_celseq2_expression_matrix',
                    4:'pancreas_multi_fluidigmc1_expression_matrix', 5:'pancreas_multi_smartseq2_expression_matrix',
                    3:'pancreas_multi_celseq_expression_matrix',
                    7:'jurkat', 6:'293t', 8:'jurkat_293t_50_50', 9:'jurkat_293t_99_1',
                    10:'68k_pbmc', 11:'b_cells', 12:'cd14_monocytes', 13:'cd4_t_helper', 14:'cd56_nk',
                    15:'cytotoxic_t', 16:'memory_t', 17:'regulatory_t', 18:'pbmc_10X', 19:'pbmc_kang',
                    20:'nuclei', 21:'Cerebellum_ALT', 22:'Cortex_noRep5_FRONTALonly', 23:'Cortex_noRep5_POSTERIORonly',
                    24:'EntoPeduncular', 25:'GlobusPallidus', 26:'Hippocampus', 27:'Striatum',
                    28:'SubstantiaNigra', 29:'Thalamus',  }
    optionmapping = {
        1: [6, 7, 8],
        2: [1, 2, 3, 4, 5],
        3: [11, 12, 13, 14, 15, 16, 17, 18],
    }

    for datasetnumber in optionmapping[datasetnumber]:
        datasetname, genecountdata, gene_array = read_real_data(datasetnumber)
        batch_label = [datasetname for x in range(genecountdata.shape[0])]
        batch_labels.extend(batch_label)
        genecountdatalist.append(genecountdata)
        gene_arraylist.append(gene_array)

    merged_counts, shared_genes = merge_datasets(genecountdatalist, gene_arraylist)
    
    # Actual cell types
    cell_labels, _, _ = read_cell_labels(datasetnumber)
    cell_annotations = pd.DataFrame(cell_labels)
    cell_annotations.columns = ['actual']
    
    # Annotations 
    gene_annotations = pd.DataFrame(index=shared_genes)
    cell_annotations['batch'] = batch_labels

    from scipy.sparse import vstack
    X = vstack(merged_counts)
    # AnnData 
    mergeddata = AnnData(X,obs=cell_annotations,var=gene_annotations)
    filename = '../{}_v2.h5ad'.format(casename)
    mergeddata.write(filename)

Prepare data to upload to Zenodo 

In [ ]:
for dataset in ['10Xmouse','humanpancreas','10Xpbmc']:
    batch = dict()
    actual = dict()
    adata = sc.read('../Data/{}_v2.h5ad'.format(dataset))
    filename = '../Data/Finalised_Data/{}_data_only.h5ad'.format(dataset)
    newadata = AnnData(adata.X,obs=adata.obs[['actual','batch']],var=adata.var)
    print(newadata)   
    newadata.write(filename)
    
for dataset in ['10Xmouse','humanpancreas','10Xpbmc']:
    batch = dict()
    actual = dict()
    adata = sc.read('../Data/{}_v4_processed.h5ad'.format(dataset))
    filename = '../Data/Finalised_Data/{}_results_only.h5ad'.format(dataset)
    adata.X = None 
    adata.write(filename)    

In [1]:
from anndata import AnnData

import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import scanpy as sc
from scipy.stats import entropy

In [14]:
for dataset in ['10Xmouse','humanpancreas','10Xpbmc']:
    batch = dict()
    actual = dict()
    filename = '../Data/Finalised_Data/{}.h5ad'.format(dataset)
    adata = sc.read(filename)
    print(adata)

AnnData object with n_obs × n_vars = 9530 × 32643
    obs: 'actual', 'batch', 'IHPF_0.001_kmeans_normalised', 'IHPF_0.001_max', 'IHPF_0.01_kmeans_normalised', 'IHPF_0.01_max', 'IHPF_0.1_kmeans_normalised', 'IHPF_0.1_max', 'IHPF_0.25_kmeans_normalised', 'IHPF_0.25_max', 'IHPF_0.5_kmeans_normalised', 'IHPF_0.5_max', 'IHPF_0.75_kmeans_normalised', 'IHPF_0.75_max', 'HPF_kmeans_normalised', 'HPF_max'
    obsm: 'HPF', 'IHPF_0.001', 'IHPF_0.01', 'IHPF_0.1', 'IHPF_0.25', 'IHPF_0.5', 'IHPF_0.75'
    varm: 'HPF', 'IHPF_0.001', 'IHPF_0.01', 'IHPF_0.1', 'IHPF_0.25', 'IHPF_0.5', 'IHPF_0.75'
AnnData object with n_obs × n_vars = 15921 × 15369
    obs: 'actual', 'batch', 'IHPF_0.001_kmeans_normalised', 'IHPF_0.001_max', 'IHPF_0.01_kmeans_normalised', 'IHPF_0.01_max', 'IHPF_0.1_kmeans_normalised', 'IHPF_0.1_max', 'IHPF_0.25_kmeans_normalised', 'IHPF_0.25_max', 'IHPF_0.5_kmeans_normalised', 'IHPF_0.5_max', 'IHPF_0.75_kmeans_normalised', 'IHPF_0.75_max', 'HPF_kmeans_normalised', 'HPF_max'
    obsm: 'HPF'